In [1]:
import cv2
import numpy as np

In [2]:
from matplotlib import pyplot as plt

In [3]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

## Testing basic H threshold

In [399]:
video_path = "./ignore/lab_green_screen/suyash_oneplus.mp4"
# video_path = "./ignore/lab_green_screen/ankesh_oneplus.mp4"

In [401]:
# img = cv2.imread("./image/img01.png")
# cap = cv2.VideoCapture("./video/test.mp4")
cap = cv2.VideoCapture(video_path)
# cap = cv2.VideoCapture("./ignore/lab_green_screen/ankesh_oneplus.mp4")
ret,IMG = cap.read()
IMG = cv2.resize(IMG,(IMG.shape[1]//2,IMG.shape[0]//2))

In [417]:
IMG = cv2.imread("./image/test.jpg")

In [418]:
def segment(orig,params,al,bet,gm,mult):
    h_mean,h_std,l_mean,l_std,s_mean,s_std = params
    hls_im = cv2.cvtColor(orig,cv2.COLOR_BGR2HLS)
    a = (hls_im[:,:,0]>(h_mean-mult*h_std))
    b = (hls_im[:,:,0]<(h_mean+mult*h_std))
    h_mask = a*b
    a = (hls_im[:,:,1]>(l_mean-mult*l_std))
    b = (hls_im[:,:,1]<(l_mean+mult*l_std))
    l_mask = a*b
    a = (hls_im[:,:,2]>(s_mean-mult*s_std))
    b = (hls_im[:,:,2]<(s_mean+mult*s_std))
    s_mask = a*b
    hls_mask = (al*h_mask + bet*l_mask + gm*s_mask)
#     return hls_mask
    hls_mask = hls_mask.clip(0.0,1.0)
    hls_mask = 1 - hls_mask
    return hls_mask

In [419]:
def get_params(img):
    hls_img = cv2.cvtColor(img,cv2.COLOR_BGR2HLS).astype(np.float32)
    r = cv2.selectROI("Select window of background",img=img,fromCenter=False,showCrossair=False)
    cv2.destroyAllWindows()
    r = [int(x) for x in r]
    h_mean,l_mean,s_mean = np.mean(region,axis=(0,1))
    h_std,l_std,s_std = np.std(region,axis=(0,1))
    return [h_mean,h_std,l_mean,l_std,s_mean,s_std]

In [420]:
params = get_params(IMG)

In [421]:
WHITE = np.ones(IMG.shape,dtype=np.int8)*255

In [416]:
cap = cv2.VideoCapture(video_path)
while(True):
    ret,img = cap.read()
    if not ret:
        break
    img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
    bright_img = brighten(img,1,0,0.3)
    mask = np.expand_dims(segment(bright_img,params_new,0.7,0.1,0.7,8),-1)
    mask = mod_mask(mask,0,0.2)
    new_img = (mask*img+(1-mask)*WHITE).astype(np.uint8)
#     show_images([("img",ans),("masked",new_img)])
    cv2.imshow("img",img)
    cv2.imshow("masked",new_img)
    ret = cv2.waitKey(1)
    if ret==27:
        break
cv2.destroyAllWindows()

##### alpha,beta,gamma correction

In [146]:
def brighten(img,alpha,beta,gamma):
    cor_img = img.astype(np.float32)
#     alpha = 1.3
#     beta = 40
#     gamma = 0.4
    bright_img = alpha*cor_img + beta
    gam_cor = np.power(bright_img/255,gamma)*255
    bright_img = gam_cor.clip(0,255).astype(np.uint8)
    return bright_img

In [184]:
def mod_mask(mask,low,high):
    mask = mask.copy()
    mask[mask>high]=1.0
    mask[mask<low] = 0.0
    return mask

In [422]:
bright_img = brighten(IMG,1,0,0.3)
params_new = get_params(bright_img)

In [423]:
mask = np.expand_dims(segment(bright_img,params_new,0.7,0.1,0.7,8),-1)
mask = mod_mask(mask,0,0.2)
show_images([("orig",IMG),("bright",bright_img),("mask",mask)])

In [424]:
new_img = (mask*IMG+(1-mask)*WHITE).astype(np.uint8)

In [425]:
show_images([("a",new_img)])

In [96]:
mask[mask<0.2]=0